# 競馬予測

## 0. colabの環境を整える

### 0-1. git clone

In [ ]:
!git clone https://github.com/yuugo0724/keiba_prediction.git

### 0-2. ソースコードのディレクトリに移動

In [ ]:
%cd keiba_prediction/src/

### 0-3. pythonのライブラリをインストール

In [ ]:
pip install -r ../dockerfile/requirements.txt

## 1. モジュールやライブラリのインポート

### 1-1. インポート

In [ ]:
"""
ライブラリ
"""
import os
import re
import subprocess
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
from optuna.integration import lightgbm as lgb
import lightgbm as lgb_orig

"""
モジュール(定数)
"""
# ローカルパス
from modules.constants import LocalPaths
# データフレームの列名
#from modules.constants import ResultsCols
# スクレイピングのパス
#from modules.constants import ScrapyPath

"""
モジュール(前処理)
"""
# 前処理
#from modules.preprocess import shaping
from modules.preprocess import _scrapy_data


### 1-2. ローカルパスの定義

In [ ]:
# インスタンス化(path_list => pl)
lp = LocalPaths()
# プロジェクトのベースディレクトリ
base_dir = lp.BASE_DIR
# scrapyのベースディレクトリ
scrapy_dir = lp.SCRAPY_DIR
# scrapy keibaプロジェクトのパス
scrapy_keiba_dir = lp.SCRAPY_KEIBA_DIR
# dataディレクトリ
data_dir = lp.DATA_DIR
# レース結果スクレイピング用url格納ディレクトリ
data_url_dir = lp.DATA_URL_DIR
# masterデータ格納ディレクトリ
data_master_dir = lp.DATA_MASTER_DIR
data_tmp_dir = lp.DATA_TMP_DIR
# gradesデータ格納ディレクトリ
data_grades_dir = lp.DATA_GRADES_DIR
# horse_gradesデータ格納ディレクトリ
data_horse_grades_dir = lp.DATA_HORSE_GRADES_DIR
# pedigreeデータ格納ディレクトリ
data_pedigree_dir = lp.DATA_PEDIGREE_DIR
# gradesのmaster
data_grades_master = lp.DATA_GRADES_MASTER
# horse_idのmaster
data_horse_id_master = lp.DATA_HORSE_ID_MASTER
# urlの正常性チェックプログラムのパス
#sy_unc_path = base_path + 'scrapy/url_normality_check.py'


## 2. 学習データ作成
2-1. スクレイピングを実施  
2-2. 前処理  
2-3. 学習データを説明変数と目的変数に分割  


### 2-1. スクレイピング
- スクレイピング対象のurlを取得  
- レース結果データを取得  
- 馬ごとの成績データを取得  
  ※学習データとしては、まだ利用していない  
- 血統データを取得  
  ※学習データとしては、まだ利用していない  

#### 2-1-1. 変数の定義

In [ ]:
# レース結果スクレイピング対象のurl取得
proc_coll_url = lp.PROC_COLL_URL
log_coll_url = lp.LOG_COLL_URL
# レース結果スクレイピング
proc_coll_grades = lp.PROC_COLL_GRADES
log_coll_grades = lp.LOG_COLL_GRADES
# 馬ごとのレース結果スクレイピング
proc_coll_horse_grades = lp.PROC_COLL_HORSE_GRADES
log_coll_horse_grades = lp.LOG_COLL_HORSE_GRADES
# 馬ごとの血統データスクレイピング
proc_coll_pedigree = lp.PROC_COLL_PEDIGREE
log_coll_pedigree = lp.LOG_COLL_PEDIGREE


#### 2-1-2. スクレイピング対象のurl取得

In [ ]:
df_gen_date = pd.date_range(start="20120101",end="20221101", freq="MS")
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
horse_id_list = df_date.values

os.chdir(scrapy_keiba_dir)
with open(log_coll_url, 'w') as f:
  for horse_id in horse_id_list:
    date_dir = os.path.join(data_url_dir,horse_id[0:4])
    os.makedirs(date_dir, exist_ok=True)
    scrapy_cmd = ["python3",proc_coll_url,horse_id,date_dir,data_url_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

#### 2-1-3. レース結果のパスリスト作成

In [ ]:
url_file_list = []
for current_dir, sub_dirs, files_list in os.walk(data_url_dir):
  for file in files_list:
    url_file_list.append(os.path.join(current_dir,file))

#### 2-1-4. レース結果の取得

##### 2-1-4-1. 取得対象のレース期間を指定

In [ ]:
df_gen_date = pd.date_range(start="20200101",end="20201201", freq="MS")
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
horse_id_list = df_date.values

target_url_file_list = []
for date in horse_id_list:
  date_match = '.*/' + date + '.csv'
  target_url_files = [url_file for url_file in url_file_list if re.match(date_match,url_file)]
  if target_url_files:
    target_url_file_list.extend(target_url_files)

##### 2-1-4-2. レース結果のスクレイピング

In [ ]:
os.chdir(scrapy_keiba_dir)
with open(log_coll_grades, 'w') as f:
  for url_file in target_url_file_list:
    file_name = url_file.split('/')[-1].split('.')[0]
    date_y = file_name[0:4]
    date_m = file_name[4:6]
    race_url_list = np.ravel(pd.read_csv(url_file,header=0).values.tolist())
    date_dir = os.path.join(data_grades_dir,date_y,date_m)
    os.makedirs(date_dir, exist_ok=True)
    for race_url in race_url_list:
      race_id = re.sub("\D","", race_url)
      scrapy_cmd = ["python3",proc_coll_grades,race_url,race_id,date_dir,data_grades_dir]
      scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
      scrapy_proc.wait()
os.chdir(base_dir)

# colabで取得する場合、時間がたつと切断されてしまうため、
# 取得後ブランチを作ってpushする
!git brach
!git brach -M $(date +'%Y%m%d%H%M%S')
!git add .
!git commit -m "スクレイピングの結果格納"
!git push

#### 2-1-5. 成績マスターの作成
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
race_list = []
for current_dir, sub_dirs, files_list in os.walk(data_grades_dir):
  for file in files_list:
    race_list.append(os.path.join(current_dir,file))
#print(race_list)
_scrapy_data.create_grades_master(race_list,data_grades_dir)

#### 2-1-6. 成績マスターの読み込み

In [ ]:
grades_master = pd.read_pickle(data_grades_master)

#### 2-1-7. 馬IDマスターの作成

In [ ]:
df_horse_id_master = grades_master['馬ID']
df_horse_id_master = df_horse_id_master.drop_duplicates()
df_horse_id_master = df_horse_id_master.reset_index(drop=True)
df_horse_id_master.to_pickle(data_horse_id_master)

#### 2-1-8. 馬のレース結果を取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]

os.chdir(scrapy_keiba_dir)
with open(log_coll_horse_grades, 'w') as f:
  for horse_id in horse_id_list:
    scrapy_cmd = ["python3",proc_coll_horse_grades,horse_id,data_horse_grades_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

#### 2-1-9. 血統データを取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]

os.chdir(scrapy_keiba_dir)
with open(log_coll_pedigree, 'w') as f:
  for horse_id in horse_id_list:
    scrapy_cmd = ["python3",proc_coll_pedigree,horse_id,data_horse_grades_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

### 2-2. 前処理

#### 2-2-1. 欠損値の削除

In [ ]:
grades_master = grades_master.dropna(how='any')

#### 2-2-2. 体重増減を整数化(記号を削除)

In [ ]:
grades_master['馬体重増減'] = grades_master['馬体重増減'].replace("+","").astype('int')

#### 2-2-3. 性齢の分割

In [ ]:
sexual_age = grades_master['性齢']
sex = sexual_age.replace('[0-9]+',"", regex=True)
age = sexual_age.replace("\D","", regex=True)
grades_master['性'] = sex
grades_master['齢'] = age.astype(int)

#### 2-2-4. レース名の処理※要検討

In [ ]:
grades_master['レース名'].drop_duplicates().to_csv('test.csv')

#### 2-2-5. カテゴリ変数をダミー変数化

In [ ]:
grades_master = pd.get_dummies(grades_master,columns=['レース名'])
grades_master = pd.get_dummies(grades_master,columns=['回り'])
grades_master = pd.get_dummies(grades_master,columns=['天候'])
grades_master = pd.get_dummies(grades_master,columns=['タイプ'])
grades_master = pd.get_dummies(grades_master,columns=['馬場状態'])
grades_master = pd.get_dummies(grades_master,columns=['馬名'])
grades_master = pd.get_dummies(grades_master,columns=['騎手'])
grades_master = pd.get_dummies(grades_master,columns=['調教師'])
grades_master = pd.get_dummies(grades_master,columns=['性'])

#### 2-2-6. 不要な列を削除

In [ ]:
grades_master = grades_master.drop('レースID', axis=1)
grades_master = grades_master.drop('馬番', axis=1)
grades_master = grades_master.drop('性齢', axis=1)
grades_master = grades_master.drop('タイム', axis=1)
grades_master = grades_master.drop('単勝', axis=1)
grades_master = grades_master.drop('人気', axis=1)
grades_master = grades_master.drop('馬ID', axis=1)
grades_master = grades_master.drop('調教師ID', axis=1)

#### 2-2-7. データフレームの型をintに変換
変換対象列  
- 距離  
- 枠番  
- 斥量  
- 馬体重

スクレイピングのバグで距離列に空白が含まれていたのでそちらを削除  
※現時点では修正済み

In [ ]:
grades_master = grades_master[grades_master['距離'] != '']

In [ ]:
grades_master['距離'] = grades_master['距離'].astype(int)
grades_master['枠番'] = grades_master['枠番'].astype(int)
grades_master['斥量'] = grades_master['斥量'].astype(float)
grades_master['馬体重'] = grades_master['馬体重'].astype(int)

#### 2-2-7. 3着以内とそれ以外でデータを2分類化する

In [ ]:
grades_master['着順'] = pd.to_numeric(grades_master['着順'],errors='coerce')
grades_master = grades_master.dropna(how='any', axis=0)
grades_master['着順'] = grades_master['着順'].astype('int')
grades_master.loc[grades_master['着順']<=3,['着順']] = 1
grades_master.loc[grades_master['着順']>3,['着順']] = 0

In [ ]:
print(grades_master.dtypes)

### 2-3. 学習データを説明変数と目的変数に分割

In [ ]:
#df_tran_data = df_tran_data.drop(['馬体重(増減)'], axis=1)

x = grades_master.drop(['着順'], axis=1)
y = grades_master['着順']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

#### 1-5-4. 学習データと検証データに分ける

### 1-6. 学習モデルの作成・学習

#### 1-6-1. ハイパーパラメータの設定

In [19]:
params = {
  'objective': 'binary',
  'metric': 'auc',
}
best_params, histroy ={}, []
model = lgb.train(params,
                  lgb_train,
                  valid_sets=[lgb_train,lgb_eval],
                  num_boost_round=10,
                  early_stopping_rounds=10)
best_params_ = model.params

#### 1-6-2. 学習モデル作成

In [ ]:
model = lgb_orig.train(best_params_,
                        lgb_train,
                        valid_sets=lgb_eval,
                        num_boost_round=100,
                        early_stopping_rounds=10)

###

In [ ]:
pred = x_test.sample(n=1)
#print(pred)
y_pred = model.predict(pred)
print(y_pred)